# Description

**TODO**

# Modules loading

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from IPython.display import display
from pathlib import Path

import pandas as pd

import conf

# Settings

In [3]:
# if True, then it doesn't check if result files already exist and runs everything again
FORCE_RUN = True

In [4]:
PREDICTION_METHOD = "Module-based"

In [5]:
OUTPUT_DIR = conf.RESULTS["DRUG_DISEASE_ANALYSES"]
display(OUTPUT_DIR)
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses')

In [6]:
# OUTPUT_DATA_DIR = Path(OUTPUT_DIR, "data")
# display(OUTPUT_DATA_DIR)
# OUTPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

In [7]:
INPUT_DATA_DIR = Path(
    OUTPUT_DIR,
    "data",
    "proj",
)
display(INPUT_DATA_DIR)
INPUT_DATA_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj')

In [8]:
OUTPUT_PREDICTIONS_DIR = Path(OUTPUT_DIR, "predictions")
display(OUTPUT_PREDICTIONS_DIR)
OUTPUT_PREDICTIONS_DIR.mkdir(parents=True, exist_ok=True)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions')

# Load PharmacotherapyDB gold standard

In [9]:
gold_standard = pd.read_pickle(
    Path(OUTPUT_DIR, "gold_standard.pkl"),
)

In [10]:
display(gold_standard.shape)

(998, 3)

In [11]:
display(gold_standard.head())

,trait,drug,true_class
0,DOID:10652,DB00843,1
1,DOID:10652,DB00674,1
2,DOID:10652,DB01043,1
3,DOID:10652,DB00989,1
4,DOID:10652,DB00810,0


In [12]:
doids_in_gold_standard = set(gold_standard["trait"])

# Load LINCS

## Projected data

In [13]:
input_file = Path(INPUT_DATA_DIR, "lincs-projection.pkl").resolve()

display(input_file)

PosixPath('/media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/data/proj/lincs-projection.pkl')

In [14]:
lincs_projection = pd.read_pickle(input_file)

In [15]:
display(lincs_projection.shape)

(987, 1170)

In [16]:
display(lincs_projection.head())

perturbagen,DB00014,DB00091,DB00121,DB00130,DB00131,DB00132,DB00136,DB00140,DB00146,DB00150,...,DB08995,DB09002,DB09004,DB09009,DB09010,DB09015,DB09019,DB09020,DB09022,DB09023
LV1,0.036115,-0.358610,0.091067,0.008923,0.046469,0.136747,0.036151,0.034024,-0.027445,0.060644,...,0.069930,0.021537,0.020511,-0.006250,-0.002660,-0.017210,-0.004029,-0.145490,0.102504,0.057621
LV2,0.012281,0.009738,0.004650,-0.006554,0.021758,0.010425,0.000794,-0.000603,0.011534,-0.004425,...,0.000344,0.001123,-0.012675,-0.004849,0.000375,0.002623,-0.003252,-0.066209,0.025378,0.012453
LV3,-0.005533,-0.059174,0.013454,0.008906,-0.004939,0.033663,-0.010045,0.003824,0.009814,-0.007492,...,-0.021941,-0.002417,0.000155,0.002941,-0.015100,-0.016372,0.012249,0.009216,0.018254,-0.003516
LV4,-0.004151,-0.032884,0.005934,-0.003991,-0.028524,0.002709,0.007001,0.024704,0.005058,-0.004345,...,0.013806,0.003197,0.013099,0.002643,-0.008836,0.023852,0.028957,0.014681,0.000826,-0.009739
LV5,-0.015156,-0.005276,0.025747,0.000346,-0.013932,-0.010911,0.041403,-0.018285,0.003507,-0.007621,...,-0.011577,-0.015956,-0.027614,-0.001707,-0.000542,-0.016918,0.001959,0.024147,0.013004,-0.014166


# Load S-PrediXcan

In [17]:
from entity import Trait

In [18]:
phenomexcan_input_file_list = [
    f
    for f in INPUT_DATA_DIR.glob("*.pkl")
    if f.name.startswith("spredixcan-")
    #     if f.name.startswith(("smultixcan-", "spredixcan-"))
]

In [19]:
display(len(phenomexcan_input_file_list))

49

In [20]:
pd.read_pickle(phenomexcan_input_file_list[10]).head()

,I9_PHLETHROMBDVTLOW-DVT_of_lower_extremities,I71-Diagnoses_main_ICD10_I71_Aortic_aneurysm_and_dissection,G62-Diagnoses_main_ICD10_G62_Other_polyneuropathies,2395_4-Hairbalding_pattern_Pattern_4,20003_1141168590-Treatmentmedication_code_pariet_10mg_ec_tablet,4056_raw-Age_stroke_diagnosed,M13_FOREIGNBODY-Residual_foreign_body_in_soft_tissue,20002_1494-Noncancer_illness_code_selfreported_varicose_veins,I85-Diagnoses_main_ICD10_I85_Oesophageal_varices,6158_4-Why_reduced_smoking_Financial_reasons,...,41215_0-Detention_categories_Informal_not_formally_detained,M76-Diagnoses_main_ICD10_M76_Enthesopathies_of_lower_limb_excluding_foot,22601_35633208-Job_coding_vocational_or_industrial_trainerinstructor,H7_RETINALDETACH-Retinal_detachments_and_breaks,20003_1141172918-Treatmentmedication_code_celluvisc_1_singleuse_eye_drops,22601_35643209-Job_coding_careers_adviser_or_vocational_guidance_specialist_placement_advisor,20002_1537-Noncancer_illness_code_selfreported_joint_pain,20514-Recent_lack_of_interest_or_pleasure_in_doing_things,22601_53192534-Job_coding_builder_building_contractor,30280_raw-Immature_reticulocyte_fraction
LV1,-0.014789,-0.005148,0.003527,0.044255,-0.049789,-0.003370,-0.012221,-0.044921,0.043811,-0.015323,...,0.028341,0.011670,0.006078,-0.022633,0.025303,-0.024274,-0.032986,0.034739,-0.044636,0.012119
LV2,0.034505,-0.027832,-0.000509,-0.014187,-0.005945,0.052514,-0.001453,0.024439,0.008691,-0.005137,...,-0.017043,-0.047705,0.008300,0.011851,0.023302,0.019575,-0.007766,-0.008164,-0.000387,-0.033172
LV3,0.008558,0.018979,0.027071,-0.021982,0.002781,-0.028696,-0.007911,-0.002396,-0.001959,-0.028391,...,0.004264,0.012013,-0.003572,-0.007833,0.029271,0.008884,0.001608,0.003328,0.020263,0.049979
LV4,0.006327,-0.037406,0.004182,-0.005890,0.029469,0.025135,-0.015742,-0.009342,-0.006136,-0.006324,...,-0.007909,-0.028944,-0.016226,0.025116,0.014557,-0.011344,0.049692,-0.061459,-0.020588,-0.020565
LV5,-0.018782,-0.005410,0.012185,0.060662,0.015323,0.019732,0.054043,-0.014707,0.009779,0.026989,...,-0.025060,0.001125,-0.024277,0.055004,-0.026691,-0.001697,0.040285,-0.041917,-0.027834,-0.007988


# Predict drug-disease associations

In [21]:
from drug_disease import (
#     predict_dotprod,
    predict_dotprod_neg,
#     predict_pearson,
#     predict_pearson_neg,
#     predict_spearman,
#     predict_spearman_neg,
)

In [22]:
for phenomexcan_input_file in phenomexcan_input_file_list:
    print(phenomexcan_input_file.name)

    # read phenomexcan data
    phenomexcan_projection = pd.read_pickle(phenomexcan_input_file)
    print(f"  shape: {phenomexcan_projection.shape}")

    #     predict_dotprod(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    predict_dotprod_neg(
        lincs_projection,
        phenomexcan_input_file,
        phenomexcan_projection,
        OUTPUT_PREDICTIONS_DIR,
        PREDICTION_METHOD,
        doids_in_gold_standard,
        FORCE_RUN,
    )

    #     predict_pearson(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    #     predict_pearson_neg(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    #     predict_spearman(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    #     predict_spearman_neg(
    #         lincs_projection,
    #         phenomexcan_input_file,
    #         phenomexcan_projection,
    #         OUTPUT_PREDICTIONS_DIR,
    #         PREDICTION_METHOD,
    #         doids_in_gold_standard,
    #         FORCE_RUN,
    #     )

    print("\n")

spredixcan-mashr-zscores-Adipose_Subcutaneous-projection.pkl
  shape: (987, 4091)
  predicting all_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.003725
std,0.026250
min,-1.030164
25%,-0.005382
50%,0.002967
75%,0.011277
max,1.397537


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Adipose_Subcutaneous-projection-all_genes-prediction_scores.h5


spredixcan-mashr-zscores-Adipose_Visceral_Omentum-projection.pkl
  shape: (987, 4091)
  predicting all_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.003750
std,0.025032
min,-0.891217
25%,-0.005161
50%,0.003050
75%,0.011225
max,1.159593


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Adipose_Visceral_Omentum-projection-all_genes-prediction_scores.h5


spredixcan-mashr-zscores-Adrenal_Gland-projection.pkl
  shape: (987, 4091)
  predicting all_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


...

    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Uterus-projection-all_genes-prediction_scores.h5


spredixcan-mashr-zscores-Vagina-projection.pkl
  shape: (987, 4091)
  predicting all_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.003491
std,0.023547
min,-0.846198
25%,-0.004933
50%,0.002764
75%,0.010509
max,1.216750


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Vagina-projection-all_genes-prediction_scores.h5


spredixcan-mashr-zscores-Whole_Blood-projection.pkl
  shape: (987, 4091)
  predicting all_genes...
    shape: (1170, 4091)
    shape (after DOID map): (1170, 351)
  building classifier data...
    shape: (410670, 3)


,score
count,410670.000000
mean,0.003655
std,0.025200
min,-1.090423
25%,-0.005271
50%,0.003001
75%,0.011229
max,0.843625


    saving to: /media/miltondp/Elements1/projects/phenoplier/results/drug_disease_analyses/predictions/dotprod_neg/spredixcan-mashr-zscores-Whole_Blood-projection-all_genes-prediction_scores.h5


